In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_squared_error


In [2]:
csv_file = 'wifi_data.csv'
data = pd.read_csv(csv_file)

In [ ]:

# Step 1: Read the CSV file


# Step 2: Process data by floor
floors = data['floor'].unique()

for floor in floors:
    floor_data = data[data['floor'] == floor]
    
    # Create a grid for the heatmap
    x_min, x_max = floor_data['x'].min(), floor_data['x'].max()
    y_min, y_max = floor_data['y'].min(), floor_data['y'].max()
    x_range = np.arange(x_min, x_max + 1)
    y_range = np.arange(y_min, y_max + 1)
    grid_x, grid_y = np.meshgrid(x_range, y_range)
    
    # Interpolate signal levels onto the grid
    grid_z = np.full(grid_x.shape, np.nan)
    
    for _, row in floor_data.iterrows():
        grid_z[row['y'] - y_min, row['x'] - x_min] = row['level']
    
    # Use Gaussian filter to smooth the heatmap
    grid_z = gaussian_filter(grid_z, sigma=1, mode='constant', cval=np.nan)
    
    # Step 3: Create the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(grid_z, xticklabels=x_range, yticklabels=y_range, cmap='YlGnBu', cbar_kws={'label': 'Signal Level (dBm)'})
    plt.title(f'WiFi Signal Heatmap - Floor {floor}')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.gca().invert_yaxis()  # Invert y-axis to match typical coordinate systems
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from PIL import Image

# Step 1: Read the CSV file
csv_file = 'wifi_data.csv'
data = pd.read_csv(csv_file)

# Step 2: Process data by floor
floors = data['floor'].unique()

for floor in floors:
    floor_data = data[data['floor'] == floor]
    
    # Load the floor map image
    floor_map_image = Image.open(f'floor_map_{floor}.png')
    
    # Create a grid for the heatmap
    x_min, x_max


In [4]:
wifi_df = pd.DataFrame(data)

In [5]:
pivot_df = wifi_df.pivot_table(index=['floor', 'x', 'y'], columns='BSSID', values='level', fill_value=-100)
pivot_df = pivot_df.reset_index()

In [9]:
wifi_df['level'].max()

-7

In [6]:
X = pivot_df.drop(columns=['floor', 'x', 'y'])
y = pivot_df[['floor', 'x', 'y']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

# Step 5: Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy:.2f}')

mse = mean_squared_error(y_test[['x', 'y']], y_pred[['x', 'y']])
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) for x and y coordinates: {rmse:.2f}')

floor_accuracy = accuracy_score(y_test['floor'], y_pred['floor'])
print(f'Floor prediction accuracy: {floor_accuracy:.2f}')


ValueError: multiclass-multioutput is not supported